<a href="https://colab.research.google.com/github/vsolodkyi/RecSystem_SkillBox/blob/main/module_2/Skillbox_recsys_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендательная система на библиотеке surprise (Практика 2)

Сайт библиотеки: http://surpriselib.com/

Установки библиотеки:
```
$ pip install scikit-surprise
```
или
```
$ conda install -c conda-forge scikit-surprise```

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("recdemo.csv", sep=";") #небольшой dataset

In [4]:
df

,id,A,B,C,D,E,F
0,1,3.0,4.0,2.0,4.0,1.0,NaN
1,2,3.0,4.0,2.0,4.0,NaN,2.0
2,3,NaN,2.0,5.0,5.0,NaN,5.0
3,4,NaN,NaN,NaN,NaN,4.0,NaN
4,5,3.0,NaN,2.0,4.0,4.0,NaN
5,6,NaN,5.0,5.0,5.0,NaN,5.0
6,7,1.0,NaN,NaN,2.0,NaN,3.0
7,8,NaN,NaN,NaN,NaN,4.0,4.0


In [5]:
df_unpivot=pd.melt(df, id_vars=['id'])
df_unpivot.head()

,id,variable,value
0,1,A,3.0
1,2,A,3.0
2,3,A,NaN
3,4,A,NaN
4,5,A,3.0


In [6]:
df_unpivot.dropna(inplace=True)
df_unpivot.columns=['userID', 'itemID', 'rating']

In [7]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 6.7 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633968 sha256=be5865b0a46adcf47726def84a8aec77c04ab674bc52df951141a4db21ed11b4
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [8]:
from surprise import Dataset
from surprise import Reader


In [9]:
reader = Reader(rating_scale=(1, 5)) # Зададим разброс оценок
data = Dataset.load_from_df(df_unpivot[['userID', 'itemID', 'rating']], reader) #создадим объект, с которым умеет работать библиотека

In [10]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

$$ x_{ij}=μ+b_i+b_j+q^T_jp_i $$

https://surprise.readthedocs.io/en/stable/matrix_factorization.html

In [11]:
from surprise import SVD  

In [12]:
algo = SVD(n_factors=2,random_state=999)
predictions = algo.fit(trainset).test(testset)

In [13]:
df_unpivot1 = df_unpivot.copy()
for i in predictions:
    df_unpivot1 = df_unpivot1.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)

In [14]:
df_unpivot1.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,3.312456
2,3.000000,4.000000,2.000000,4.000000,3.210599,2.000000
3,3.444350,2.000000,5.000000,5.000000,3.684146,5.000000
4,3.242607,3.599635,3.415230,3.784069,4.000000,3.647434
5,3.000000,3.490772,2.000000,4.000000,4.000000,3.533499
6,3.676756,5.000000,5.000000,5.000000,3.884204,5.000000
7,1.000000,3.194893,2.985395,2.000000,3.010203,3.000000
8,3.301363,3.630884,3.442314,3.798689,4.000000,4.000000


In [15]:
algo.pu # пользователи

array([[-0.02356038,  0.086795  ],
       [ 0.03311668, -0.06586878],
       [-0.01663012, -0.05196951],
       [ 0.17326108, -0.22112997],
       [ 0.11922751,  0.0574848 ],
       [-0.03847042, -0.02430361],
       [-0.10099924, -0.04709509],
       [-0.00974106,  0.08383197]])

In [16]:
algo.qi # фильмы

array([[-0.0499725 ,  0.20377982],
       [ 0.07260589, -0.09175883],
       [-0.0048234 , -0.06959711],
       [-0.08501758, -0.10890099],
       [ 0.16616968,  0.1825492 ],
       [ 0.00912318, -0.15003955]])

In [17]:
algo.qi.T # фильмы

array([[-0.0499725 ,  0.07260589, -0.0048234 , -0.08501758,  0.16616968,
         0.00912318],
       [ 0.20377982, -0.09175883, -0.06959711, -0.10890099,  0.1825492 ,
        -0.15003955]])

In [18]:
mu=df_unpivot.rating.mean()

In [19]:
m= algo.pu @ algo.qi.T 

In [20]:
m.shape

(8, 6)

In [21]:
i = 0
j = 5

In [22]:
mu+m[i][j]+algo.bu[i]+algo.bi[j]

3.31245613193024

## Сравнение параметров

In [23]:
from surprise import KNNBaseline
from surprise.model_selection import cross_validate

В библиотеке есть встроенные Dataset. Можно воспользоваться ими.

Возьмем Dataset movielens (Подробнее о нём https://grouplens.org/datasets/movielens/ )

In [24]:
data = Dataset.load_builtin('ml-100k')##загружем данные о фильмах

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [25]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user', 'item', 'rating', 'timestamp']
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [26]:
for i in [1,3,5,7,20]:
    algo = KNNBaseline(k=i, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'NN:',np.mean(cv['test_rmse']))

1NN: 1.2451860795907113
3NN: 1.041396033068743
5NN: 0.9928217236390958
7NN: 0.9710432333510898
20NN: 0.9406048369864716


In [27]:
for i in [1,2,3,5,7,10,50,70]:
    algo = SVD(n_factors=i,random_state=999, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'-factors:',np.mean(cv['test_rmse']))

1-factors: 0.9439006231015004
2-factors: 0.9453816984752729
3-factors: 0.9443402457161398
5-factors: 0.9419912564965712
7-factors: 0.9414820218439074
10-factors: 0.9427793674145893
50-factors: 0.9431736391149929
70-factors: 0.9444084319882262


$$ x_{ij}=q^T_jp_i $$

https://surprise.readthedocs.io/en/stable/matrix_factorization.html

In [28]:
for i in [1,2,3,5,7,10,50,70]:
    algo = SVD(n_factors=i,biased=False,random_state=999, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'-factors:',np.mean(cv['test_rmse']))

1-factors: 0.9665244763884254
2-factors: 0.9605920203675744
3-factors: 0.962584621207912
5-factors: 0.9597799038520227
7-factors: 0.9546432059906759
10-factors: 0.9552126273420288
50-factors: 0.9554923532961742
70-factors: 0.9613287136288259


## Задание:
```
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.30, random_state=999)
```
1. Для параметров i = [1,5,9,12,17,22] найдите лучший SVD (random_state=999) алгоритм по метрике MAP(n=7).

2. Сделайте прогноз по всем пользователям, который равен среднему из KNN (k=15, c корреляцией Пирсона и item-based подходом) и SVD (n = лучший из пункта 1).
Сравните по метрике MAP(n=7) полученные предсказания с kNN и SVD.

In [43]:
from surprise.model_selection import train_test_split

In [44]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.30, random_state=999)

In [45]:
def Avg_Precision_at_n(df_ratings, n=10,threshold=5):
    avg_p=0
    rel_n = set(df_ratings[df_ratings.real_rating>=threshold]['itemID'])
    
    for i in range(1,n+1):
        t=df_ratings.sort_values(by='rating', ascending=False)[0:i]
        ret_n=set(t['itemID'])
        t2=df_ratings.sort_values(by='rating', ascending=False)[i-1:i]
        ret_n2=set(t2['itemID'])
        #print(ret_n2,ret_n)
        if len((ret_n2 & rel_n))>0:
            avg_p=avg_p+len(rel_n & ret_n)/n
    return avg_p/n

def MAP_at_n(df_ratings_all, n=10,threshold=5):
    users=df_ratings_all['userID'].unique()
    map_at_n=0
    for i in users:
        df_ratings=df_ratings_all[df_ratings_all.userID==i]
        map_at_n=map_at_n+Avg_Precision_at_n(df_ratings,n=n,threshold=threshold)
    return map_at_n/len(users)  

In [46]:
for i in [1,5,9,12,17,22]:
    algo = SVD(n_factors=i,random_state=999, verbose=False) #отключим вывод логирования
    predictions = algo.fit(trainset).test(testset)
    #algo_user = KNNBaseline(k=k, verbose=False, sim_options = {'user_based': False})
    df_ratings = pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
    for i in predictions:
      #if int(i.uid)==833:
      df_ratings = df_ratings.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)

    print(MAP_at_n(df_ratings, n=7))

0.11792585538987567
0.11985197048066282
0.11985197048066278
0.11889973380656582
0.12229748739368461
0.11768779622135146


the best model is SVD with n factors = 17

## 2. Сделайте прогноз по всем пользователям, который равен среднему из KNN (k=15, c корреляцией Пирсона и item-based подходом) и SVD (n = лучший из пункта 1). Сравните по метрике MAP(n=7) полученные предсказания с kNN и SVD.

In [47]:
from surprise import KNNBaseline

In [48]:
knn = KNNBaseline(k=15, sim_options = {'name': 'pearson_baseline', 'used_based': False}, vervbose=False)
svd = SVD(n_factors=17, random_state=999, verbose=False)

In [49]:
predictions = knn.fit(trainset).test(testset)

df_ratings = pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
for i in predictions:
  df_ratings = df_ratings.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [50]:
df_ratings.head()

,userID,itemID,rating,real_rating
0,753,357,3.978457,4.0
1,542,585,3.147980,2.0
2,5,364,2.764920,1.0
3,887,385,4.027836,4.0
4,383,480,4.626760,5.0


In [58]:
predictions2 = svd.fit(trainset).test(testset)

df_ratings2 = pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
for i in predictions2:
  df_ratings2 = df_ratings2.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)

In [59]:
df_ratings2.head()

,userID,itemID,rating,real_rating
0,753,357,3.543399,4.0
1,542,585,2.627620,2.0
2,5,364,2.345794,1.0
3,887,385,4.106974,4.0
4,383,480,4.550769,5.0


In [60]:
df_knn_svd = df_ratings.merge(df_ratings2[['userID', 'itemID', 'rating']], how='left', on=['userID', 'itemID'])

In [61]:
df_knn_svd['rating'] = (df_knn_svd['rating_x'] + df_knn_svd['rating_y']) / 2

In [62]:
df_knn_svd.head()

,userID,itemID,rating_x,real_rating,rating_y,rating
0,753,357,3.978457,4.0,3.543399,3.760928
1,542,585,3.147980,2.0,2.627620,2.887800
2,5,364,2.764920,1.0,2.345794,2.555357
3,887,385,4.027836,4.0,4.106974,4.067405
4,383,480,4.626760,5.0,4.550769,4.588765


In [64]:
print(MAP_at_n(df_ratings,7), MAP_at_n(df_ratings2,7), MAP_at_n(df_knn_svd, 7))

0.12210271171034662 0.12229748739368461 0.12621464280303815
